In [1]:
import pandas as pd

In [2]:
vcf_file = "variants.vcf.gz"
vcf_file1 = "GAsP_1163Samples.SNVs.Feb2020.vcf.gz"
vcf_file2 = r"I:\Genomic Project\GAsP_1163Samples.InDels.Feb2020.vcf.gz"
vcf_file3 = r"D:\genome_asia_main_file\genome_asia_data\Multiallelic\GAsP_Multiallelic_Y_SNP_bgzip.vcf.gz"

In [3]:
def get_vcf_header(vcf_file_input):
    import gzip
    header_line = ""
    # set the opener if it is gzip
    opener = gzip.open if vcf_file_input.endswith(".gz") else open
    # open the file and find the header line
    with opener(vcf_file_input, "rt") as file:
        for line in file:
            if line.startswith("#CHROM"):
                header_line = line
                break
            elif not line.startswith("#"):
                break
    # format the header line to a list format
    if header_line:
        header_columns = header_line.strip("#").strip("\n").split("\t")
        return header_columns
    else:
        return "Unable to find the header"

fixed_columns = get_vcf_header(vcf_file3)  # change file name here >>
print(f"These are the columns found :\n{fixed_columns}\n")


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\genome_asia_main_file\\genome_asia_data\\Multiallelic\\GAsP_Multiallelic_Y_SNP_bgzip.vcf.gz'

In [ ]:
df = pd.read_csv(vcf_file3, sep='\t', comment='#', names=fixed_columns)
# if csv file uncomment below and comment above
# df = pd.read_csv(vcf_file3, sep=",")
df.head(10)


## T2T MAF Calculation Logic

In [ ]:
# Finding MAF and writing in standard format to INFO column
import re

# FREQ=dbGaP_PopFreq:1,0;GNO;RS=2068592700;SSR=0;VC=DEL
expression=r"dbGaP_PopFreq:([^;]+)"

def extract_maf(row):
    if 'dbGaP_PopFreq:' in row:
        for part in row.split(';'):
            match=re.search(expression, part)
            if match:
                values = match.group(1).split(',')
                try:
                    # listing all the ALT freq
                    alt_freqs = [float(v) for v in values if v not in ('.', '0', '1')]
                    #print(alt_freqs)
                    # Find the MAF min of ALT
                    maf = min(alt_freqs)
                    #print(maf)
                    return f"MAF={maf:.6g}"

                except ValueError:
                    pass
    return "MAF=0"

T2T_SNP_MAF_0.05.csv
def extract_maf2(row):
    if 'dbGaP_PopFreq:' in row:
        for part in row.split(';'):
            match=re.search(expression, part)
            if match:
                values = match.group(1).split(',')
                try:
                    # listing all the ALT freq
                    alt_freqs = [float(v) for v in values if v not in ('.', '0', '1')]
                    #print(alt_freqs)
                    # Find the MAF min of ALT
                    maf = min(alt_freqs)
                    #print(maf)
                    return maf

                except ValueError:
                    pass

    # if nothing found
    return 0

# Testing
extract_maf("FREQ=dbGaP_PopFreq:1,0;GNO;RS=2068592700;SSR=0;VC=DEL")
extract_maf2("FREQ=dbGaP_PopFreq:1,0;GNO;RS=2068592700;SSR=0;VC=DEL")


In [ ]:
# Apply extract_maf to the INFO col
df['INFO'] = df['INFO'] + ';' + df['INFO'].apply(extract_maf)
# And add a new col
df['MAF'] = df['INFO'].apply(extract_maf2)


In [ ]:
df

### Filtering

In [ ]:
filt = df['MAF'] > 0.05
df[filt].tail()

In [ ]:
# File saving to vcf file add the header ##INFO=<ID=MAF,Number=1,Type=Float,Description="Minor Allele Frequency">

# Save to a csv file
df_save = df[filt]
df_save.to_csv(r"D:\genome_asia_main_file\genome_asia_data\T2T\T2T_SNP_B_MAF_0.05.csv", index=False)